# LINEAR CLASSIFICATION AND LOGISTIC REGRESSION

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('C:\\Users\\HP\\Downloads\\hamoye\\dataset\\NFA 2019 public_data.csv')

df.head()

c:\users\hp\downloads\python\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [3]:
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [4]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [5]:
df=df.dropna()
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51713 entries, 0 to 72185
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         51713 non-null  object 
 1   year            51713 non-null  int64  
 2   country_code    51713 non-null  int64  
 3   record          51713 non-null  object 
 4   crop_land       51713 non-null  float64
 5   grazing_land    51713 non-null  float64
 6   forest_land     51713 non-null  object 
 7   fishing_ground  51713 non-null  float64
 8   built_up_land   51713 non-null  float64
 9   carbon          51713 non-null  float64
 10  total           51713 non-null  float64
 11  QScore          51713 non-null  object 
dtypes: float64(6), int64(2), object(4)
memory usage: 5.1+ MB


In [7]:
df['QScore'].value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [8]:
df['QScore']=df['QScore'].replace(['1A'],'2A')
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [9]:
df_2A=df[df.QScore=='2A']
df_3A=df[df.QScore=='3A'].sample(350)
data_df=df_2A.append(df_3A)

import sklearn.utils
data_df=sklearn.utils.shuffle(data_df)
data_df=data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()


3A    350
2A    240
Name: QScore, dtype: int64

In [63]:
#more preprocessing

X = data_df.drop(columns='QScore')
y = data_df['QScore']

#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()


3A    249
2A    164
Name: QScore, dtype: int64

In [66]:
#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
x_train.record=encoder.fit_transform(x_train.record)
x_test.record=encoder.transform(x_test.record)

import imblearn
from imblearn.over_sampling import SMOTE
smote=SMOTE(random_state=1)
x_train_balanced,y_balanced=smote.fit_sample(x_train,y_train)

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
normalized_train_df=scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalized_train_df=pd.DataFrame(normalized_train_df,columns=x_train_balanced.drop(columns=['record']).columns)
normalized_train_df['record']=x_train_balanced['record']

x_test=x_test.reset_index(drop=True)
normalized_test_df=scaler.transform(x_test.drop(columns=['record']))
normalized_test_df=pd.DataFrame(normalized_test_df,columns=x_test.drop(columns=['record']).columns)
normalized_test_df['record']=x_test['record']

#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(normalized_train_df ,y_balanced)

c:\users\hp\downloads\python\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


LogisticRegression()

# Measuring Classification Performance


Cross-validation and accuracy

In [14]:
from sklearn.model_selection import cross_val_score
scores= cross_val_score(log_reg,normalized_train_df,y_balanced,cv=5,scoring='f1_macro')
scores

array([0.44      , 0.51515152, 0.47812124, 0.4244186 , 0.48146246])

confusion matrix

In [67]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions=log_reg.predict(normalized_test_df)
cnf_mat=confusion_matrix(y_true=y_test,y_pred=new_predictions,labels=['2A','3A'])
cnf_mat

array([[39, 37],
       [51, 50]], dtype=int64)

# METRICS

Accuracy

In [68]:
accuracy=accuracy_score(y_true=y_test,y_pred=new_predictions)
print('Accuracy:{}'.format(round(accuracy*100,2)))

Accuracy:50.28


Precision

In [69]:
precision=precision_score(y_true=y_test,y_pred=new_predictions,pos_label='2A')
print('Precision:{}'.format(round(precision*100,2)))

Precision:43.33


Recall

In [70]:
Recall=recall_score(y_true=y_test,y_pred=new_predictions,pos_label='2A')
print('Recall:{}'.format(round(Recall*100,2)))

Recall:51.32


In [ ]:
F1-score

In [71]:
f1=f1_score(y_true=y_test,y_pred=new_predictions,pos_label='2A')
print('f1:{}'.format(round(f1*100,2)))

f1:46.99


K-Fold Cross Validation

In [88]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalized_train_df) 
f1_scores = []
#run for every split
for train_index, test_index in kf.split(normalized_train_df):
    x_train, x_test = normalized_train_df.iloc[train_index],normalized_train_df.iloc[test_index]
    y_train, y_test = y_balanced[train_index],y_balanced[test_index]
model = LogisticRegression().fit(x_train, y_train)
  #save result to list
f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A')*100)
f1_scores

[0.0]

Stratified K-Fold Cross Validation

In [76]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=1)
f1_scores=[]
for train_index,test_index in skf.split(normalized_train_df,y_balanced):
    x_train,x_test=np.array(normalized_train_df)[train_index],np.array(normalized_train_df)[test_index]
    y_train,y_test=y_balanced[train_index],y_balanced[test_index]
    model=LogisticRegression().fit(x_train,y_train)
    f1_scores.append(f1_score(y_true=y_test,y_pred=model.predict(x_test),pos_label='2A'))
f1_scores

[0.4285714285714286,
 0.6451612903225806,
 0.4536082474226804,
 0.40425531914893614,
 0.5]

Leave One Out Cross Validation (LOOCV)

In [82]:
from sklearn.model_selection import LeaveOneOut
loo=LeaveOneOut()
scores=cross_val_score(LogisticRegression(),normalized_train_df,y_balanced,cv=loo,scoring='f1_macro')
average_score=scores.mean()*100
average_score

48.393574297188756

Tree-Based Methods and The Support Vector Machine

In [89]:
from sklearn.tree import DecisionTreeClassifier
dec_tree=DecisionTreeClassifier()
dec_tree.fit(normalized_train_df,y_balanced)

DecisionTreeClassifier()

In [3]:
import numpy as np
import pandas as pd
import graphviz
from sklearn import tree
from sklearn.metrics import accuracy_score
data=pd.read_csv('C:\\Users\\HP\\Downloads\\hamoye\\dataset\\Data_for_UCI_named.csv', delimiter = ',')
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [6]:
print(data.shape)
data[0:]
data.columns

(10000, 14)


Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stab', 'stabf'],
      dtype='object')

In [9]:
input=np.zeros((10000,13))

In [10]:
input[0:,0]=data['tau1']
input[:,1]=data['tau2']
input[:,2]=data['tau3']
input[:,3]=data['tau4']
input[:,4]=data['p1']
input[:,5]=data['p2']
input[:,6]=data['p3']
input[:,7]=data['p4']
input[:,8]=data['g1']
input[:,9]=data['g2']
input[:,10]=data['g3']
input[:,11]=data['g4']
input[:,12]=data['stab']

In [11]:
output=np.zeros((10000,0))
output=data['stabf']


In [ ]:
train=input